In [2]:
import pandas as pd
import requests
import io

#download from Github

url = "https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv"

download = requests.get(url).content

df = pd.read_csv(io.StringIO(download.decode('utf-8')))

df.head()

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
0,2016,2016-07-09,7921,FA Women's Super League,Liverpool Women,Reading,51.56,50.42,0.4389,0.2767,...,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,2016-07-10,7921,FA Women's Super League,Arsenal Women,Notts County Ladies,46.61,54.03,0.3572,0.3608,...,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,2016-07-10,7921,FA Women's Super League,Chelsea FC Women,Birmingham City,59.85,54.64,0.4799,0.2487,...,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,2016-07-16,7921,FA Women's Super League,Liverpool Women,Notts County Ladies,53.00,52.35,0.4289,0.2699,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2016,2016-07-17,7921,FA Women's Super League,Chelsea FC Women,Arsenal Women,59.43,60.99,0.4124,0.3157,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#filter down to 2022 and EPL
df2 = df[(df['season'] == 2022) & (df['league_id']==2411)]

df2 = df2.reset_index(drop=True)

df2.head()

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
0,2022,2022-08-05,2411,Barclays Premier League,Crystal Palace,Arsenal,71.36,78.53,0.3159,0.4205,...,22.8,48.6,0.0,2.0,1.30,1.36,1.40,2.41,0.0,1.72
1,2022,2022-08-06,2411,Barclays Premier League,Fulham,Liverpool,58.70,90.51,0.0926,0.7466,...,39.7,68.2,2.0,2.0,1.25,2.57,0.78,1.41,2.1,2.10
2,2022,2022-08-06,2411,Barclays Premier League,Tottenham Hotspur,Southampton,81.87,61.33,0.7059,0.1182,...,60.3,33.6,4.0,1.0,1.91,0.54,1.88,1.23,4.2,1.05
3,2022,2022-08-06,2411,Barclays Premier League,Leeds United,Wolverhampton,62.15,67.72,0.3941,0.3456,...,33.5,23.5,2.0,1.0,0.84,1.11,1.02,2.58,2.1,1.05
4,2022,2022-08-06,2411,Barclays Premier League,Newcastle,Nottingham Forest,68.65,57.31,0.5401,0.2045,...,30.4,42.6,2.0,0.0,1.82,0.26,3.33,0.48,1.9,0.00


In [4]:
#get week numbers that count up by every 10 games using the index of the PL sheet as to account for the gap in the season
df2['week_num'] = (df2.index / 10 + 1).astype(int)
df2.head()

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2,week_num
0,2022,2022-08-05,2411,Barclays Premier League,Crystal Palace,Arsenal,71.36,78.53,0.3159,0.4205,...,48.6,0.0,2.0,1.30,1.36,1.40,2.41,0.0,1.72,1
1,2022,2022-08-06,2411,Barclays Premier League,Fulham,Liverpool,58.70,90.51,0.0926,0.7466,...,68.2,2.0,2.0,1.25,2.57,0.78,1.41,2.1,2.10,1
2,2022,2022-08-06,2411,Barclays Premier League,Tottenham Hotspur,Southampton,81.87,61.33,0.7059,0.1182,...,33.6,4.0,1.0,1.91,0.54,1.88,1.23,4.2,1.05,1
3,2022,2022-08-06,2411,Barclays Premier League,Leeds United,Wolverhampton,62.15,67.72,0.3941,0.3456,...,23.5,2.0,1.0,0.84,1.11,1.02,2.58,2.1,1.05,1
4,2022,2022-08-06,2411,Barclays Premier League,Newcastle,Nottingham Forest,68.65,57.31,0.5401,0.2045,...,42.6,2.0,0.0,1.82,0.26,3.33,0.48,1.9,0.00,1


In [5]:
#split table by t1 an t2. plan is to split these, drop the numbers, re merge on opp stats and then append both together
dft1 = df2[['season','date','team1','spi1','prob1','probtie','proj_score1','importance1','score1','xg1','nsxg1','adj_score1','week_num']]

dft2 = df2[['season','date','team2','spi2','prob2','probtie','proj_score2','importance2','score2','xg2','nsxg2','adj_score2','week_num']]

dft2.head()

,season,date,team2,spi2,prob2,probtie,proj_score2,importance2,score2,xg2,nsxg2,adj_score2,week_num
0,2022,2022-08-05,Arsenal,78.53,0.4205,0.2636,1.39,48.6,2.0,1.36,2.41,1.72,1
1,2022,2022-08-06,Liverpool,90.51,0.7466,0.1607,2.38,68.2,2.0,2.57,1.41,2.10,1
2,2022,2022-08-06,Southampton,61.33,0.1182,0.1758,0.82,33.6,1.0,0.54,1.23,1.05,1
3,2022,2022-08-06,Wolverhampton,67.72,0.3456,0.2604,1.26,23.5,1.0,1.11,2.58,1.05,1
4,2022,2022-08-06,Nottingham Forest,57.31,0.2045,0.2554,0.84,42.6,0.0,0.26,0.48,0.00,1


In [6]:
#rename column names
dft12 = dft1.rename(columns={'team1' : 'team', 'spi1' : 'spi', 'prob1' : 'prob_w', 'proj_score1' : 'proj_score', 'importance1' : 'importance', 'score1' : 'score', 'xg1' : 'xg', 'nsxg1' : 'nsxg', 'adj_score1' : 'adj_score'})

dft22 = dft2.rename(columns={'team2' : 'team', 'spi2' : 'spi', 'prob2' : 'prob_w', 'proj_score2' : 'proj_score', 'importance2' : 'importance', 'score2' : 'score', 'xg2' : 'xg', 'nsxg2' : 'nsxg', 'adj_score2' : 'adj_score'})
dft22.head()

,season,date,team,spi,prob_w,probtie,proj_score,importance,score,xg,nsxg,adj_score,week_num
0,2022,2022-08-05,Arsenal,78.53,0.4205,0.2636,1.39,48.6,2.0,1.36,2.41,1.72,1
1,2022,2022-08-06,Liverpool,90.51,0.7466,0.1607,2.38,68.2,2.0,2.57,1.41,2.10,1
2,2022,2022-08-06,Southampton,61.33,0.1182,0.1758,0.82,33.6,1.0,0.54,1.23,1.05,1
3,2022,2022-08-06,Wolverhampton,67.72,0.3456,0.2604,1.26,23.5,1.0,1.11,2.58,1.05,1
4,2022,2022-08-06,Nottingham Forest,57.31,0.2045,0.2554,0.84,42.6,0.0,0.26,0.48,0.00,1


In [7]:
#grab opp stats. dft1op contains the opponent stats for whoever t1 was playing (t2). then rename those to be called opp_

dft1op = dft2[['team2','spi2','prob2','proj_score2','importance2','score2','xg2','nsxg2','adj_score2']]

dft1op2 = dft1op.rename(columns={'team2' : 'opp_name', 'spi2' : 'opp_spi', 'prob2' : 'prob_l', 'proj_score2' : 'opp_proj_score', 'importance2' : 'opp_importance', 'score2' : 'opp_score', 'xg2' : 'opp_xg', 'nsxg2' : 'opp_nsxg', 'adj_score2' : 'opp_adj_score'})

dft2op = dft1[['team1','spi1','prob1','proj_score1','importance1','score1','xg1','nsxg1','adj_score1']]

dft2op2 = dft2op.rename(columns={'team1' : 'opp_name', 'spi1' : 'opp_spi', 'prob1' : 'prob_l', 'proj_score1' : 'opp_proj_score', 'importance1' : 'opp_importance', 'score1' : 'opp_score', 'xg1' : 'opp_xg', 'nsxg1' : 'opp_nsxg', 'adj_score1' : 'opp_adj_score'})

dft2op2.head()

,opp_name,opp_spi,prob_l,opp_proj_score,opp_importance,opp_score,opp_xg,opp_nsxg,opp_adj_score
0,Crystal Palace,71.36,0.3159,1.16,22.8,0.0,1.30,1.40,0.0
1,Fulham,58.70,0.0926,0.71,39.7,2.0,1.25,0.78,2.1
2,Tottenham Hotspur,81.87,0.7059,2.29,60.3,4.0,1.91,1.88,4.2
3,Leeds United,62.15,0.3941,1.37,33.5,2.0,0.84,1.02,2.1
4,Newcastle,68.65,0.5401,1.54,30.4,2.0,1.82,3.33,1.9


In [8]:
#merge dft1 and dft1op2
print(len(dft12.index))
print(len(dft1op2.index))

dft1wopp = dft12.merge(dft1op2, how='inner', left_index=True, right_index=True)

print(len(dft1wopp))

dft1wopp.head()

380
380
380


,season,date,team,spi,prob_w,probtie,proj_score,importance,score,xg,...,week_num,opp_name,opp_spi,prob_l,opp_proj_score,opp_importance,opp_score,opp_xg,opp_nsxg,opp_adj_score
0,2022,2022-08-05,Crystal Palace,71.36,0.3159,0.2636,1.16,22.8,0.0,1.30,...,1,Arsenal,78.53,0.4205,1.39,48.6,2.0,1.36,2.41,1.72
1,2022,2022-08-06,Fulham,58.70,0.0926,0.1607,0.71,39.7,2.0,1.25,...,1,Liverpool,90.51,0.7466,2.38,68.2,2.0,2.57,1.41,2.10
2,2022,2022-08-06,Tottenham Hotspur,81.87,0.7059,0.1758,2.29,60.3,4.0,1.91,...,1,Southampton,61.33,0.1182,0.82,33.6,1.0,0.54,1.23,1.05
3,2022,2022-08-06,Leeds United,62.15,0.3941,0.2604,1.37,33.5,2.0,0.84,...,1,Wolverhampton,67.72,0.3456,1.26,23.5,1.0,1.11,2.58,1.05
4,2022,2022-08-06,Newcastle,68.65,0.5401,0.2554,1.54,30.4,2.0,1.82,...,1,Nottingham Forest,57.31,0.2045,0.84,42.6,0.0,0.26,0.48,0.00


In [9]:
#merge dft2 and dft2op2

print(len(dft22.index))
print(len(dft2op2.index))

dft2wopp = dft22.merge(dft2op2, how='inner', left_index=True, right_index=True)

print(len(dft2wopp))

dft2wopp.head()

380
380
380


,season,date,team,spi,prob_w,probtie,proj_score,importance,score,xg,...,week_num,opp_name,opp_spi,prob_l,opp_proj_score,opp_importance,opp_score,opp_xg,opp_nsxg,opp_adj_score
0,2022,2022-08-05,Arsenal,78.53,0.4205,0.2636,1.39,48.6,2.0,1.36,...,1,Crystal Palace,71.36,0.3159,1.16,22.8,0.0,1.30,1.40,0.0
1,2022,2022-08-06,Liverpool,90.51,0.7466,0.1607,2.38,68.2,2.0,2.57,...,1,Fulham,58.70,0.0926,0.71,39.7,2.0,1.25,0.78,2.1
2,2022,2022-08-06,Southampton,61.33,0.1182,0.1758,0.82,33.6,1.0,0.54,...,1,Tottenham Hotspur,81.87,0.7059,2.29,60.3,4.0,1.91,1.88,4.2
3,2022,2022-08-06,Wolverhampton,67.72,0.3456,0.2604,1.26,23.5,1.0,1.11,...,1,Leeds United,62.15,0.3941,1.37,33.5,2.0,0.84,1.02,2.1
4,2022,2022-08-06,Nottingham Forest,57.31,0.2045,0.2554,0.84,42.6,0.0,0.26,...,1,Newcastle,68.65,0.5401,1.54,30.4,2.0,1.82,3.33,1.9


In [10]:
#append both tables together

dffinal = pd.concat([dft1wopp, dft2wopp])

dffinal.head()

,season,date,team,spi,prob_w,probtie,proj_score,importance,score,xg,...,week_num,opp_name,opp_spi,prob_l,opp_proj_score,opp_importance,opp_score,opp_xg,opp_nsxg,opp_adj_score
0,2022,2022-08-05,Crystal Palace,71.36,0.3159,0.2636,1.16,22.8,0.0,1.30,...,1,Arsenal,78.53,0.4205,1.39,48.6,2.0,1.36,2.41,1.72
1,2022,2022-08-06,Fulham,58.70,0.0926,0.1607,0.71,39.7,2.0,1.25,...,1,Liverpool,90.51,0.7466,2.38,68.2,2.0,2.57,1.41,2.10
2,2022,2022-08-06,Tottenham Hotspur,81.87,0.7059,0.1758,2.29,60.3,4.0,1.91,...,1,Southampton,61.33,0.1182,0.82,33.6,1.0,0.54,1.23,1.05
3,2022,2022-08-06,Leeds United,62.15,0.3941,0.2604,1.37,33.5,2.0,0.84,...,1,Wolverhampton,67.72,0.3456,1.26,23.5,1.0,1.11,2.58,1.05
4,2022,2022-08-06,Newcastle,68.65,0.5401,0.2554,1.54,30.4,2.0,1.82,...,1,Nottingham Forest,57.31,0.2045,0.84,42.6,0.0,0.26,0.48,0.00


In [96]:
#dffinal.to_csv(r'C:\Users\Jon\Downloads\testjoin.csv')
#checked that dates match across. Also checked that team names in one iteration match opp_team

In [11]:
dffinal['comp_goal'] = ((dffinal['adj_score'] + dffinal['xg'] + dffinal['nsxg']) / 3).astype(float)
dffinal['opp_comp_goal'] = ((dffinal['opp_adj_score'] + dffinal['opp_xg'] + dffinal['opp_nsxg']) / 3).astype(float)

dffinal.head()

,season,date,team,spi,prob_w,probtie,proj_score,importance,score,xg,...,opp_spi,prob_l,opp_proj_score,opp_importance,opp_score,opp_xg,opp_nsxg,opp_adj_score,comp_goal,opp_comp_goal
0,2022,2022-08-05,Crystal Palace,71.36,0.3159,0.2636,1.16,22.8,0.0,1.30,...,78.53,0.4205,1.39,48.6,2.0,1.36,2.41,1.72,0.900000,1.830000
1,2022,2022-08-06,Fulham,58.70,0.0926,0.1607,0.71,39.7,2.0,1.25,...,90.51,0.7466,2.38,68.2,2.0,2.57,1.41,2.10,1.376667,2.026667
2,2022,2022-08-06,Tottenham Hotspur,81.87,0.7059,0.1758,2.29,60.3,4.0,1.91,...,61.33,0.1182,0.82,33.6,1.0,0.54,1.23,1.05,2.663333,0.940000
3,2022,2022-08-06,Leeds United,62.15,0.3941,0.2604,1.37,33.5,2.0,0.84,...,67.72,0.3456,1.26,23.5,1.0,1.11,2.58,1.05,1.320000,1.580000
4,2022,2022-08-06,Newcastle,68.65,0.5401,0.2554,1.54,30.4,2.0,1.82,...,57.31,0.2045,0.84,42.6,0.0,0.26,0.48,0.00,2.350000,0.246667


In [12]:
dffinal.to_csv(r'C:\Users\Jon\Downloads\eplstats.csv', index=False)